# pysentimiento: A transformer-based Sentiment Analysis library for Spanish

En esta notebook mostramos un breve ejemplo de cómo usar [pysentimiento](https://github.com/pysentimiento/pysentimiento/), un toolkit multilingual para extracción de opiniones y análisis de sentimientos (aunque centrado en el idioma español)

`pysentimiento` es un una librería que utiliza modelos pre-entrenados de [transformers](https://github.com/huggingface/transformers) para distintas tareas de SocialNLP. Usa como modelos bases a [BETO](https://github.com/dccuchile/beto) y [RoBERTuito](https://github.com/pysentimiento/robertuito) en Español, y BERTweet en inglés.

-- 

In this notebook we show a brief example of how to use [pysentimiento](https://github.com/pysentimiento/pysentimiento/), a multilingual toolkit for opinion mining and sentiment analysis.

`pysentimiento` is a library that uses pre-trained models of [transformers] (https://github.com/huggingface/transformers) for different SocialNLP tasks. It uses as base models [BETO] (https://github.com/dccuchile/beto) and [RoBERTuito] (https://github.com/pysentimiento/robertuito) in Spanish, and BERTweet in English.

 
First, let's install the library

In [1]:
!pip install pysentimiento

You should consider upgrading via the '/home/jmperez/.cache/pypoetry/virtualenvs/pysentimiento-bwlKzHxB-py3.7/bin/python -m pip install --upgrade pip' command.


Let's create an analyzer. The `create_analyzer` receives the task and the language as parameters (currently supports "es" and "en").

In [2]:
from pysentimiento import create_analyzer
analyzer = create_analyzer(task="sentiment", lang="es")


Veamos algunos ejemplos:

In [3]:
analyzer.predict("Qué gran jugador es Messi")

AnalyzerOutput(output=POS, probas={POS: 0.994, NEG: 0.003, NEU: 0.003})

In [4]:
analyzer.predict("Esto es pésimo")

AnalyzerOutput(output=NEG, probas={NEG: 0.948, NEU: 0.048, POS: 0.004})

In [5]:
analyzer.predict("Qué es esto?")

AnalyzerOutput(output=NEU, probas={NEU: 0.802, NEG: 0.188, POS: 0.010})

### Predicción en batch

Si tenemos un conjunto de oraciones, `pysentimiento` hace la predicción en conjunto de manera eficiente

In [6]:
%%time
from tqdm.auto import tqdm
oraciones = [
    "Qué gran jugador es Messi",
    "Esto es pésimo",
    "No sé, cómo se llama?",    
] * 100
for sent in tqdm(oraciones):
    analyzer.predict(sent)

  0%|          | 0/300 [00:00<?, ?it/s]

CPU times: user 2.75 s, sys: 16 ms, total: 2.76 s
Wall time: 2.74 s


In [7]:
%%time
rets = analyzer.predict(oraciones)

  0%|          | 0/10 [00:00<?, ?ba/s]

The following columns in the test set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text.
***** Running Prediction *****
  Num examples = 300
  Batch size = 32


CPU times: user 892 ms, sys: 404 ms, total: 1.3 s
Wall time: 1.18 s


### Emojis

It supports the use of emojis through the [emoji](https://pypi.org/project/emoji/) library.

Soporta también el uso de emojis

In [8]:
analyzer.predict("🤢")

AnalyzerOutput(output=NEG, probas={NEG: 0.981, NEU: 0.017, POS: 0.002})

O de hashtags

In [9]:
analyzer.predict("#EstoEsUnaMierda")

AnalyzerOutput(output=NEG, probas={NEG: 0.998, NEU: 0.001, POS: 0.001})

## Emotion Analysis

`pysentimiento` provee análisis de emociones a través de modelos pre-entrenados con los datasets de [EmoEvent](https://github.com/fmplaza/EmoEvent-multilingual-corpus/)

In [10]:
emotion_analyzer = create_analyzer(task="emotion", lang="en")

loading configuration file https://huggingface.co/finiteautomata/bertweet-base-emotion-analysis/resolve/main/config.json from cache at /home/jmperez/.cache/huggingface/transformers/c246eed05359b1a49c45955b0265b488e35b0cbd2628e3ead7dd54c8815162ee.a2dff24b4e0a884c6d58a09968c5b68e7391e749eb698ad92541818d420fd01b
Model config RobertaConfig {
  "_name_or_path": "vinai/bertweet-base",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "others",
    "1": "joy",
    "2": "sadness",
    "3": "anger",
    "4": "surprise",
    "5": "disgust",
    "6": "fear"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "anger": 3,
    "disgust": 5,
    "fear": 6,
    "joy": 1,
    "others": 0,
    "sadness": 2,
    "sur

In [11]:
emotion_analyzer.predict("This is so terrible...")

AnalyzerOutput(output=sadness, probas={sadness: 0.978, fear: 0.013, disgust: 0.003, others: 0.002, surprise: 0.002, anger: 0.001, joy: 0.001})

In [12]:
emotion_analyzer.predict("omg")

AnalyzerOutput(output=surprise, probas={surprise: 0.982, others: 0.007, fear: 0.003, joy: 0.003, sadness: 0.002, anger: 0.002, disgust: 0.001})

In [13]:
emotion_analyzer.predict("yayyyy")

AnalyzerOutput(output=joy, probas={joy: 0.879, others: 0.106, surprise: 0.005, anger: 0.005, sadness: 0.002, disgust: 0.002, fear: 0.002})

In [14]:
emotion_analyzer.predict("People in the world is really worried because of Coronavirus")

AnalyzerOutput(output=fear, probas={fear: 0.939, others: 0.043, surprise: 0.005, joy: 0.004, disgust: 0.004, sadness: 0.002, anger: 0.002})

## Hate Speech

In [15]:
hate_speech_analyzer = create_analyzer(task="hate_speech", lang="es")

loading file https://huggingface.co/pysentimiento/robertuito-hate-speech/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/pysentimiento/robertuito-hate-speech/resolve/main/special_tokens_map.json from cache at /home/jmperez/.cache/huggingface/transformers/29bbf6eb913fdd3d8aaabeec43896b21a574167c27d3a9a85fc9dc57eea39ace.0dc5b1041f62041ebbd23b1297f2f573769d5c97d8b7c28180ec86b8f6185aa8
loading file https://huggingface.co/pysentimiento/robertuito-hate-speech/resolve/main/tokenizer_config.json from cache at /home/jmperez/.cache/huggingface/transformers/f151cdb0362e2cc8c419bd5878232bea313009ac93445c0f66bf43ea764cd71c.50a2bcf7668df2ff5a82b7b0455533bb4c0db21e6e33565fa20fd7dc8a3be740
loading file https://huggingface.co/pysentimiento/robertuito-hate-speech/resolve/main/tokenizer.json from cache at /home/jmperez/.cache/huggingface/transformers/82bf0643b37bd3be5cb16b6646358a7e6c7de8829f035e79cdc14ad408ebf3ce.0843b07596b388e054bae078721182b4846b9e28a7bbf04d7079b

In [17]:
hate_speech_analyzer.predict("Esto es una mierda pero no es odio")

AnalyzerOutput(output=[], probas={hateful: 0.022, targeted: 0.009, aggressive: 0.018})

In [18]:
hate_speech_analyzer.predict("Esto es odio porque los inmigrantes deben ser aniquilados")

AnalyzerOutput(output=['hateful'], probas={hateful: 0.835, targeted: 0.008, aggressive: 0.476})

In [29]:
hate_speech_analyzer.predict("Vaya guarra barata y de poca monta es Miley Cyrus!")

AnalyzerOutput(output=['hateful', 'targeted', 'aggressive'], probas={hateful: 0.987, targeted: 0.978, aggressive: 0.969})

## Preprocessing

`pysentimiento` tiene un módulo de preprocesamiento de tweets con varias opciones para manipular hashtags, emojis, repetición de caracteres y demás.

In [15]:
from pysentimiento.preprocessing import preprocess_tweet

preprocess_tweet("📢 @realDonaldTrump ha sido banneado de Twitter #BreakingNews")

'emoji altavoz de mano emoji  @usuario ha sido banneado de Twitter breaking news'